In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from tensorflow.keras.layers import Conv2D

from preprocessing.data.imagenet_labels import imagenet_labels

In [3]:
from models.ResBlock import ResBlock

In [4]:
from modules.TFR_load import TFR_load

In [5]:
# Hyper params

BATCH_SIZE     = 32
NUM_OF_CLASS   = 10
LEARNING_RATE  = 1e-5
COSINE_DECAY   = True
NUM_TRAIN_DATA = 60000
NUM_TEST_DATA  = 10000
EPOCH = 100

TR_STEPS_PER_EPOCH = NUM_TRAIN_DATA//BATCH_SIZE
TE_STEPS_PER_EPOCH = NUM_TEST_DATA//BATCH_SIZE

In [6]:
import tensorflow_datasets as tfds

ds_name = 'mnist'
builder = tfds.builder(ds_name)

tr_ds, te_ds = builder.as_dataset(split = ['train', 'test'], shuffle_files = True)

tr_ds = tr_ds.batch(BATCH_SIZE)
print(len(tr_ds))
te_ds = te_ds.batch(BATCH_SIZE)
print(len(te_ds))

1875
313


In [7]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.conv = Conv2D(
            16,
            (5, 5),
            strides = (1, 1),
            padding='same',
            activation = "relu")
        
        self.ResBlock_1 = ResBlock(out_channel = 32, pooling = 'max')
        self.ResBlock_2 = ResBlock(out_channel = 64, pooling = 'max')
        self.ResBlock_3 = ResBlock(out_channel = 128, pooling = 'max')
        self.ResBlock_4 = ResBlock(out_channel = 256, pooling = 'max')
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(NUM_OF_CLASS, activation = 'softmax')
    def call(self, x):
        x = self.conv(x)
        x = self.ResBlock_1(x)
        x = self.ResBlock_2(x)
        x = self.ResBlock_3(x)
        x = self.ResBlock_4(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x

model = MyModel()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=1e-08)
loss = tf.keras.losses.CategoricalCrossentropy()

for _ in range(EPOCH):
    for example in tr_ds:
        images, labels = example["image"], example["label"]
        with tf.GradientTape() as tape:
            logits = model(images / 255)
            loss_value = loss(logits, tf.one_hot(labels, 10))
        print(loss_value.numpy())
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

14.506287
14.506276
14.506247
14.506235
14.506259
14.506212
14.506388
14.506207
14.506363
14.506248
14.506292
14.506285
14.5063505
14.506207
14.506343
14.50624
14.506256
14.506277
14.506186
14.506247
14.506172
14.506069
14.506457
14.506409
14.506264
14.506319
